In [1]:
!pip install ipython-sql
%load_ext sql
import os
USER = os.environ['POSTGRESQL_USER']
PASSWORD = os.environ['POSTGRESQL_PASSWORD']
POSTGRESQL_HOST = '10.129.0.25'
DBASE_NAME = 'demo'
CONNECT_DATA = 'postgresql://{}:{}@{}/{}'.format(
    USER,
    PASSWORD,
    POSTGRESQL_HOST,
    DBASE_NAME
)

In [47]:
#ticket_no  0005432312164, find: who and where flew
#Provide solutions in 3 ways:
#- a query with subqueries (nested queries)
#- a query with no subqueries and no joins
#- a query with joins

**1. A query with subqueries (nested queries)**

In [56]:
%%sql $CONNECT_DATA
SELECT passenger_id, passenger_name, b.ticket_no, arrival_airport, departure_airport
FROM Flights f
JOIN (SELECT passenger_id, passenger_name, a.ticket_no, flight_id 
     FROM Ticket_flights 
     JOIN (SELECT passenger_id, passenger_name, ticket_no 
          FROM Tickets
          WHERE ticket_no = '0005432312164') as a
      ON a.ticket_no = Ticket_flights.ticket_no) as b
ON b.flight_id = f.flight_id


4 rows affected.


passenger_id,passenger_name,ticket_no,arrival_airport,departure_airport
0423 646487,ALEKSEY GERASIMOV,0005432312164,EGO,DME
0423 646487,ALEKSEY GERASIMOV,0005432312164,EGO,AAQ
0423 646487,ALEKSEY GERASIMOV,0005432312164,DME,EGO
0423 646487,ALEKSEY GERASIMOV,0005432312164,AAQ,EGO


**2. A query with no subqueries and no joins**

In [10]:
%%sql $CONNECT_DATA
SELECT passenger_id, passenger_name, s.ticket_no, departure_airport, arrival_airport
FROM Tickets s, Ticket_flights tf, Flights f
WHERE s.ticket_no = tf.ticket_no
AND tf.flight_id = f.flight_id
AND s.ticket_no = '0005432312164'

4 rows affected.


passenger_id,passenger_name,ticket_no,departure_airport,arrival_airport
0423 646487,ALEKSEY GERASIMOV,0005432312164,DME,EGO
0423 646487,ALEKSEY GERASIMOV,0005432312164,AAQ,EGO
0423 646487,ALEKSEY GERASIMOV,0005432312164,EGO,DME
0423 646487,ALEKSEY GERASIMOV,0005432312164,EGO,AAQ


**3. A query with joins**

In [12]:
%%sql $CONNECT_DATA
SELECT passenger_id, passenger_name, s.ticket_no, departure_airport, arrival_airport
FROM Tickets s
JOIN Ticket_flights USING (ticket_no) 
JOIN Flights USING (flight_id)
WHERE s.ticket_no = '0005432312164'

4 rows affected.


passenger_id,passenger_name,ticket_no,departure_airport,arrival_airport
0423 646487,ALEKSEY GERASIMOV,0005432312164,DME,EGO
0423 646487,ALEKSEY GERASIMOV,0005432312164,AAQ,EGO
0423 646487,ALEKSEY GERASIMOV,0005432312164,EGO,DME
0423 646487,ALEKSEY GERASIMOV,0005432312164,EGO,AAQ


**Extended solution about Mrs. 'ELLA DMITRIEVA'** 

In [34]:
%%sql $CONNECT_DATA
SELECT 
passenger_name,
flight_id,
a.city as departure_city, 
b.city as arrival_city
FROM Flights f
JOIN Ticket_flights USING (flight_id)
JOIN Tickets USING (ticket_no)
JOIN Airports a
ON f.departure_airport = a.airport_code
JOIN Airports b
ON f.arrival_airport = b.airport_code
WHERE passenger_name = 'ELLA DMITRIEVA'


7 rows affected.


passenger_name,flight_id,departure_city,arrival_city
ELLA DMITRIEVA,11445,Moscow,Khanty-Mansiysk
ELLA DMITRIEVA,142605,Khanty-Mansiysk,Moscow
ELLA DMITRIEVA,34407,Moscow,Yekaterinburg
ELLA DMITRIEVA,70804,Yekaterinburg,Moscow
ELLA DMITRIEVA,74285,Yekaterinburg,Magnetiogorsk
ELLA DMITRIEVA,193132,Magnetiogorsk,Yekaterinburg
ELLA DMITRIEVA,8202,Moscow,Yuzhno-Sakhalinsk


**Aircraft model/Total number of seats ordered by the number of seats**

In [43]:
%%sql $CONNECT_DATA
SELECT model as Aircraft_model, COUNT(seat_no) as Number_of_seats_in_the_model
FROM Seats s, Aircrafts a
WHERE s.aircraft_code = a.aircraft_code
GROUP BY model
ORDER BY Number_of_seats_in_the_model desc

9 rows affected.


aircraft_model,number_of_seats_in_the_model
Boeing 777-300,402
Boeing 767-300,222
Airbus A321-200,170
Airbus A320-200,140
Boeing 737-300,130
Airbus A319-100,116
Sukhoi Superjet-100,97
Bombardier CRJ-200,50
Cessna 208 Caravan,12


In [ ]:
#Bonus - count with fare conditions 

In [46]:
%%sql $CONNECT_DATA
SELECT model as Aircraft_model, fare_conditions, COUNT(seat_no) as Number_of_seats_in_the_model
FROM Seats s, Aircrafts a
WHERE s.aircraft_code = a.aircraft_code
GROUP BY model, fare_conditions
ORDER BY Number_of_seats_in_the_model desc

17 rows affected.


aircraft_model,fare_conditions,number_of_seats_in_the_model
Boeing 777-300,Economy,324
Boeing 767-300,Economy,192
Airbus A321-200,Economy,142
Airbus A320-200,Economy,120
Boeing 737-300,Economy,118
Airbus A319-100,Economy,96
Sukhoi Superjet-100,Economy,85
Bombardier CRJ-200,Economy,50
Boeing 777-300,Comfort,48
Boeing 777-300,Business,30
